In [205]:
# DIR_PATH = "/cpt_training/app"

In [206]:
# cd {DIR_PATH}

In [207]:
# %pip install -U "setuptools<58" # setuptoolsのverが58以上だと、xfeatの依存パッケージml-metricsのinstallに失敗するため
# %pip install -q optuna xfeat japanize-matplotlib
# %pip install -q optuna

In [208]:
# import
import re

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
sns.set(font="IPAexGothic")

from glob import glob
from functools import partial
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold
from xfeat import SelectCategorical, LabelEncoder, LambdaEncoder, Pipeline, ConcatCombination, SelectNumerical, \
    ArithmeticCombinations, TargetEncoder, aggregation, GBDTFeatureSelector, GBDTFeatureExplorer
# RandomForestを使う時
from sklearn.ensemble import RandomForestRegressor

# パラメータチューニングで必要なライブラリ
import optuna
from sklearn.metrics import mean_absolute_error

In [209]:
pd.set_option('display.max_columns', None)

In [210]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
time_col = '取引時点_enc'
group_col = '都道府県名'

In [211]:
BASE_PATH = './'

In [212]:
paths = glob(BASE_PATH + 'input/train/*.csv')
train_dfs = []
for path in paths:
    train_df = pd.read_csv(path)
    train_dfs.append(train_df)
train_df = pd.concat(train_dfs)
train_df.reset_index(drop=True, inplace=True)
test_df = pd.read_csv(BASE_PATH + 'input/test.csv')

# train_df, test_df, val_dfをくっつける
master_df = pd.concat([train_df, test_df])
master_df.sort_values('取引時点', inplace=True)
master_df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_95/2232468179.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)
/tmp/ipykernel_95/2232468179.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(path)


In [213]:
master_df.columns

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

In [214]:
set(test_df['取引時点'])

{'2020年第１四半期', '2020年第２四半期'}

In [215]:
sub_df = pd.read_csv(BASE_PATH + 'input/sample_submission.csv')

In [216]:
master_df.shape

(656817, 28)

In [217]:
master_df.head(20)

,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,土地の形状,間口,延床面積（㎡）,建築年,建物の構造,用途,今後の利用目的,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,13117957,中古マンション等,NaN,13104,東京都,新宿区,大久保,東新宿,NaN,１Ｋ,20,NaN,NaN,NaN,平成17年,ＳＲＣ,住宅,NaN,NaN,NaN,NaN,商業地域,80.0,600.0,2005年第３四半期,未改装,NaN,7.342423
1,13297424,中古マンション等,NaN,13114,東京都,中野区,弥生町,中野坂上,8,１Ｋ,15,NaN,NaN,NaN,昭和59年,ＲＣ,住宅,NaN,NaN,NaN,NaN,近隣商業地域,80.0,400.0,2005年第３四半期,未改装,NaN,6.633468
2,13208440,中古マンション等,NaN,13110,東京都,目黒区,碑文谷,都立大学,NaN,１Ｋ,20,NaN,NaN,NaN,平成17年,ＲＣ,住宅,NaN,NaN,NaN,NaN,第２種住居地域,60.0,300.0,2005年第３四半期,NaN,NaN,7.301030
3,13356109,中古マンション等,NaN,13120,東京都,練馬区,大泉学園町,大泉学園,30分?60分,３ＬＤＫ,70,NaN,NaN,NaN,平成3年,ＲＣ,住宅,NaN,NaN,NaN,NaN,第１種低層住居専用地域,50.0,100.0,2005年第３四半期,改装済,NaN,7.322219
4,23181954,中古マンション等,NaN,23104,愛知県,名古屋市西区,名西,浅間町,NaN,４ＬＤＫ,85,NaN,NaN,NaN,平成13年,ＳＲＣ,住宅,NaN,NaN,NaN,NaN,商業地域,80.0,400.0,2005年第３四半期,未改装,NaN,7.361728
5,13347201,中古マンション等,NaN,13119,東京都,板橋区,志村,志村三丁目,4,３ＬＤＫ,70,NaN,NaN,NaN,昭和61年,ＲＣ,住宅,NaN,NaN,NaN,NaN,第１種住居地域,60.0,200.0,2005年第３四半期,改装済,NaN,7.361728
6,14260293,中古マンション等,NaN,14118,神奈川県,横浜市都筑区,茅ケ崎南,センター南,10,２ＬＤＫ,80,NaN,NaN,NaN,昭和64年,ＲＣ,住宅,NaN,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2005年第３四半期,未改装,NaN,7.531479
7,23177670,中古マンション等,NaN,23102,愛知県,名古屋市東区,徳川,高岳,NaN,３ＬＤＫ,155,NaN,NaN,NaN,平成3年,ＲＣ,住宅,NaN,NaN,NaN,NaN,商業地域,80.0,500.0,2005年第３四半期,改装済,NaN,7.633468
8,14261435,中古マンション等,NaN,14131,神奈川県,川崎市川崎区,浅田,八丁畷,29,NaN,50,NaN,NaN,NaN,平成3年,ＲＣ,住宅,NaN,NaN,NaN,NaN,第２種住居地域,60.0,200.0,2005年第３四半期,未改装,調停・競売等,6.939519
9,27216711,中古マンション等,NaN,27117,大阪府,大阪市旭区,高殿,関目高殿,15,２ＬＤＫ,50,NaN,NaN,NaN,昭和49年,ＳＲＣ,住宅,NaN,NaN,NaN,NaN,準工業地域,60.0,200.0,2005年第３四半期,未改装,NaN,6.886491


In [218]:
class Preprocessor:
    TARGET = TARGET
    group_col = group_col
    time_col = time_col

    def __init__(self, df):
        self.df = df.copy()
        self.agg_dfs = []
        self.labeling()

    @staticmethod
    def normalize_moyori(moyori):
        if moyori == moyori:
            if moyori == '30分?60分':
                moyori = 45
            elif moyori == '1H?1H30':
                moyori = 75
            elif moyori == '1H30?2H':
                moyori = 105
            elif moyori == '2H?':
                moyori = 120
            moyori = int(float(moyori))
        return moyori

    @staticmethod
    def normalize_area(area):
        if area == area:
            area = int(re.sub('m\^2未満|㎡以上', '', str(area)))
        return area

    @staticmethod
    def convert_wareki_to_seireki(wareki):
        if wareki == wareki:
            if wareki == '戦前':
                wareki = '昭和20年'
            value = wareki[2:-1]
            if value == '元':
                value = 1
            else:
                value = int(value)
            if '昭和' in wareki:
                seireki = 1925+value
            elif '平成' in wareki:
                seireki = 1988+value
            elif '令和' in wareki:
                seireki = 2018+value
        else:
            seireki = wareki
        return seireki

    @staticmethod
    def allocate(data):
        if pd.isna(data) or data == "":
            return 0
        else:
            if 'LDK' in data:
                return 1
            elif 'DK' in data:
                return 2
            elif 'K' in data:
                return 3
            elif 'フリースペース' in data:
                return 4
            else:
                return 5

    @staticmethod
    def extract_room_count(s):
        if pd.isna(s) or s == '':
            return 0
        numbers = re.findall(r'\d+', s)
        return int(numbers[0]) if numbers else 0

    def name_to_number(self):
        """取引時点を数字に変更する関数"""
        enc_dic = {}
        for i, e in enumerate(sorted(list(set(self.df['取引時点'].values)))):
            enc_dic[e] = i
        self.df['取引時点_enc'] = self.df['取引時点'].map(enc_dic)
        return self.df

    def remove_data(self):
        """全て同じ値になっていたり、すべてnullになっているものは削除する!"""
        rm_cols = [col for col in self.df.columns if self.df[col].nunique() <= 1]
        self.df.drop(columns=rm_cols, inplace=True)
        self.df.dropna(axis=1, how='all')

    def calc_mean_target(self):
        """各取引時点,都道府県での各都道府県のTARGETの平均値"""
        @staticmethod
        def calc_te(row):
            if row[self.time_col] in te_dic and row[self.group_col] in te_dic[row[self.time_col]]:
                try:
                    value = te_dic[row[self.time_col]-2][row[self.group_col]]
                except IndexError as e:
                    value = 0
                return value
            else:
                return 0
        te_dic = {}
        for i in set(self.df[self.time_col].values):
            tmp_df = self.df[self.df[self.time_col] < i]
            te_dic[i] = tmp_df.groupby(self.group_col)[TARGET].agg('mean').to_dict()
        # 最後にdfにapplyする
        self.df[self.group_col + '_te'] = self.df.apply(self.calc_te, axis=1)
        return self.df

    def get_agg_df(self):
        """group_valuesそれぞれのagg_methosを計算してくれるやつ"""
        agg_df, agg_cols = aggregation(self.df,
                            group_key=self.group_col,
                            group_values=['最寄駅：距離（分）'],
                            agg_methods=['count', 'mean', 'min', 'max'],
                            )
        return agg_df[agg_cols]

    def labeling(self):
        """データを切り取って渡してくれる"""
        self.df.sort_values('取引時点', inplace=True)
        self.df.reset_index(drop=True, inplace=True)
        self.df.loc[self.df['取引時点'].str.contains('2019年第３四半期|2019年第４四半期', regex=True), 'label'] = 1
        self.df.loc[self.df['取引時点'].str.contains('2020年第１四半期|2020年第２四半期', regex=True), 'label'] = 2
        self.df.loc[self.df['label'].isnull(), 'label'] = 0
        # self.feat_cols = [col for col in self.df.columns if col not in rm_cols+[ID, TARGET]]
        # self.df.dropna(how='any', axis=0)

    @staticmethod
    def checkStringExistence(input='', word=''):
        if not isinstance(input, str):
            return 0
        
        if word in input:
            return 1
        else:
            return 0
        
    def dummy(self, dummy_column):
        df_dummies = pd.get_dummies(self.df[dummy_column], prefix=dummy_column, drop_first=True)
        self.df = pd.concat([self.df, df_dummies], axis=1)
        self.df.drop(columns=dummy_column, inplace=True)

    def fit(self):
        self.remove_data()
        self.df['取引時点_何年前'] = self.df['取引時点'].apply(lambda x: 2020-int(x[:4]))
        self.name_to_number()
        # self.df.drop(columns=['取引時点'], axis=1, inplace=True)
        self.df['部屋数'] = self.df['間取り'].apply(self.extract_room_count)
        self.df['部屋タイプ'] = self.df['間取り'].apply(lambda x: self.allocate(x))
        self.df['建築年'] = self.df['建築年'].apply(lambda x: self.convert_wareki_to_seireki(x))
        self.df['面積（㎡）'] = self.df['面積（㎡）'].apply(lambda x: self.normalize_area(x))
        self.df['最寄駅：距離（分）'] = self.df['最寄駅：距離（分）'].apply(lambda x: self.normalize_moyori(x))
        self.df['心理的瑕疵の有無'] = self.df['取引の事情等'].apply(lambda input: self.checkStringExistence(input, '瑕疵有りの可能性')) # "瑕疵有りの可能性"を別のカラムに追加、次のコードで別の列から削除
        self.df['取引の事情等'].replace({'調停・競売等、瑕疵有りの可能性': '調停・競売等',
                                   '関係者間取引、瑕疵有りの可能性': '関係者間取引',
                                   '瑕疵有りの可能性': np.nan}, inplace=True)
        self.dummy('都市計画')
        self.df['最寄駅：距離（分）'] = self.df['最寄駅：距離（分）'].fillna(self.df['最寄駅：距離（分）'].mean())
        self.df['建築年'] = self.df['建築年'].fillna(self.df['建築年'].mode()[0])
        self.df.drop(columns=['容積率（％）', '建ぺい率（％）'], inplace=True)

    def transform(self):
        num_df = SelectNumerical().fit_transform(self.df)
        encoder = Pipeline([
            SelectCategorical(),
            LabelEncoder(output_suffix=""),
            ])
        le_df = encoder.fit_transform(self.df)
        self.agg_dfs.append(self.get_agg_df())
        self.preprocessed_df = pd.concat([num_df,le_df]+self.agg_dfs, axis=1)
        return self.preprocessed_df

In [219]:
# master_dfを処理していく
processor = Preprocessor(master_df)
processor.fit()
preprocessed_df = processor.transform()



In [220]:
# データの割り振り
train_df = preprocessed_df.loc[preprocessed_df['label']==0, :]
train_x = train_df.drop(columns=['label', TARGET])
# train_x.dropna(how='any', axis=0)
train_y = train_df[TARGET]

# データの割り振り
val_df = preprocessed_df.loc[preprocessed_df['label']==1, :]
val_x = val_df.drop(columns=['label', TARGET])
# val_x.dropna(how='any', axis=0)
val_y = val_df[TARGET]

# データの割り振り
test_df = preprocessed_df.loc[preprocessed_df['label']==2, :]
test_x = test_df.drop(columns=['label', TARGET])
# test_x.dropna(how='any', axis=0)
test_y = test_df[TARGET]

In [221]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 612443 entries, 0 to 612442
Data columns (total 42 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   ID                               612443 non-null  int64  
 1   市区町村コード                          612443 non-null  int64  
 2   最寄駅：距離（分）                        612443 non-null  float64
 3   面積（㎡）                            612443 non-null  int64  
 4   建築年                              612443 non-null  float64
 5   取引価格（総額）_log                     612443 non-null  float64
 6   label                            612443 non-null  float64
 7   取引時点_何年前                         612443 non-null  int64  
 8   取引時点_enc                         612443 non-null  int64  
 9   部屋数                              612443 non-null  int64  
 10  部屋タイプ                            612443 non-null  int64  
 11  心理的瑕疵の有無                         612443 non-null  int64  
 12  都市計画_工業

モデル構築

In [222]:
train_set = lgb.Dataset(train_x, label=train_y)
val_set = lgb.Dataset(val_x, label=val_y)

In [223]:
def objective(trial):
  params = {
      "objective": "regression",
      "metric": "mae",
      "boosting_type": "gbdt",
      "verbosity": -1,
      "boost_from_average": "false",
      "random_seed": 42,
      "feature_pre_filter": False,
      "max_depth": trial.suggest_int('max_depth', 4, 8),
      "num_leaves": trial.suggest_int("num_leaves", 2, 100),
      "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1),
      "feature_fraction": trial.suggest_float("feature_fraction", 0.1, 1.0),
      "bagging_fraction": trial.suggest_float("bagging_fraction", 0.1, 1.0),
      "min_child_samples": trial.suggest_int("min_child_samples", 1, 25),
      "min_data_in_leaf": trial.suggest_int('min_data_in_leaf', 1, 4)
      }

  model = lgb.train(params, train_set, valid_sets=[val_set])

  pred_y = model.predict(val_x, num_iteration=model.best_iteration)
  mae = mean_absolute_error(val_y, pred_y)

  return mae

In [224]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

[I 2024-02-04 00:19:16,030] A new study created in memory with name: no-name-487b4129-d5b1-4ccc-96f0-eadf2c7080b3
[I 2024-02-04 00:36:11,452] Trial 0 finished with value: 0.11349170126880592 and parameters: {'max_depth': 6, 'num_leaves': 42, 'learning_rate': 0.09149674012699228, 'feature_fraction': 0.21633168044662607, 'bagging_fraction': 0.5357706918709868, 'min_child_samples': 19, 'min_data_in_leaf': 3}. Best is trial 0 with value: 0.11349170126880592.
[I 2024-02-04 00:36:13,647] Trial 1 finished with value: 5.141120045806057 and parameters: {'max_depth': 6, 'num_leaves': 69, 'learning_rate': 0.003507770286431003, 'feature_fraction': 0.62692047792993, 'bagging_fraction': 0.5292069146557382, 'min_child_samples': 21, 'min_data_in_leaf': 2}. Best is trial 0 with value: 0.11349170126880592.
[I 2024-02-04 00:36:14,970] Trial 2 finished with value: 1.132145928568467 and parameters: {'max_depth': 6, 'num_leaves': 81, 'learning_rate': 0.01869951322501557, 'feature_fraction': 0.15467217112008

In [225]:
lgb_best_params = study.best_params
lgb_model = lgb.train(lgb_best_params, train_set, valid_sets=[val_set]) # early_stopping_roundsは使えなくなったそう。

[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=7 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=7 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=7 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029081 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1840
[LightGBM] [Info] Number of data points in the train set: 612443, number of used features: 40
[LightGBM] [Info] Start training from score 7.214606


In [226]:
def objective(trial):
    params = {
      "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
      "max_depth": trial.suggest_int("max_depth", 4, 20, log=True),
      "min_samples_split": trial.suggest_int("min_samples_split", 2, 14),
      "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 14),
      "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"])
      }
    
    model = RandomForestRegressor(random_state=0, **params)

    model.fit(train_x, train_y)
    pred_y = model.predict(val_x)
    mae = mean_absolute_error(val_y, pred_y)
    
    return mae

In [227]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

[I 2024-02-04 00:37:51,025] A new study created in memory with name: no-name-3613cafa-8443-4a8d-afbc-78ca64051be1
[I 2024-02-04 00:45:19,954] Trial 0 finished with value: 0.11466352409262227 and parameters: {'n_estimators': 777, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'log2'}. Best is trial 0 with value: 0.11466352409262227.
[I 2024-02-04 00:52:06,727] Trial 1 finished with value: 0.13009407362706024 and parameters: {'n_estimators': 818, 'max_depth': 11, 'min_samples_split': 12, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.11466352409262227.
[I 2024-02-04 00:53:49,242] Trial 2 finished with value: 0.17773099524530353 and parameters: {'n_estimators': 441, 'max_depth': 5, 'min_samples_split': 11, 'min_samples_leaf': 12, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.11466352409262227.
[I 2024-02-04 00:57:46,854] Trial 3 finished with value: 0.10482264207334109 and parameters: {'n_estimators': 339, 'max_depth

In [231]:
rf_best_params = study.best_params
rf_model = RandomForestRegressor(random_state=0, **rf_best_params)

In [233]:
# predict
lgb_pred = lgb_model.predict(test_x, num_iteration=lgb_model.best_iteration)
rf_model.fit(train_x, train_y)
rf_pred = rf_model.predict(test_x)
test_pred = (lgb_pred + rf_pred) / 2 # ひとまずわからんから平均とってみた

In [ ]:
# lgb.plot_importance(model, figsize=(12,8), max_num_features=50, importance_type='gain')
# plt.tight_layout()
# plt.savefig(BASE_PATH + 'output/feature_importance.png')
# plt.show()
# plt.close()

In [ ]:
test_df[TARGET] = test_pred

In [ ]:
sub_df = pd.merge(sub_df[['ID']], test_df[['ID', TARGET]], on='ID')
sub_df.to_csv(BASE_PATH + 'output/test_submission.csv', index=False)